In [48]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, metrics

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

import matplotlib.pyplot as plt
import os

import cv2

In [49]:
# Versions of each library
print(f"Tensorflow version: {tf.__version__}")
print(f"Keras version: {tf.keras.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"OpenCV version: {cv2.__version__}")

Tensorflow version: 2.8.0
Keras version: 2.8.0
Matplotlib version: 3.5.1
OpenCV version: 4.9.0


In [2]:
train_path = './images/train'
validation_path = './images/val'

In [3]:
# Initialize the data generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_path,  # Assuming train_dir is defined
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')  # Change for multi-class

validation_generator = test_datagen.flow_from_directory(
        validation_path,  # Assuming validation_dir is defined
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')  # Change for multi-class

Found 8184 images belonging to 3 classes.
Found 965 images belonging to 3 classes.


In [29]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(3, activation='softmax')  # Change for three classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=metrics.AUC(multi_label=True))

In [38]:
class_weights = {0: 1.,  # weight for class 0 (Benign)
                 1: 1.,  # weight for class 1 (Malignant)
                 2: 5.}  # weight for class 2 (Normal, under-represented)

history = model.fit(
      train_generator,
      steps_per_epoch=100,  # Depends on your dataset size
      epochs=20,
      validation_data=validation_generator,
      validation_steps=50,
      class_weight=class_weights
      ) # Depends on your dataset size

Epoch 1/20


2024-02-16 16:18:12.740565: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


100/100 [==============================] - 15s 141ms/step - loss: 0.4376 - auc_1: 0.9298 - val_loss: 0.3993 - val_auc_1: 0.9753
Epoch 2/20
100/100 [==============================] - 12s 120ms/step - loss: 0.4430 - auc_1: 0.9286
Epoch 3/20
100/100 [==============================] - 12s 118ms/step - loss: 0.3940 - auc_1: 0.9373
Epoch 4/20
100/100 [==============================] - 12s 118ms/step - loss: 0.4158 - auc_1: 0.9376
Epoch 5/20
100/100 [==============================] - 12s 120ms/step - loss: 0.4002 - auc_1: 0.9412
Epoch 6/20
100/100 [==============================] - 12s 120ms/step - loss: 0.3549 - auc_1: 0.9581
Epoch 7/20
100/100 [==============================] - 12s 118ms/step - loss: 0.3394 - auc_1: 0.9616
Epoch 8/20
100/100 [==============================] - 12s 120ms/step - loss: 0.3300 - auc_1: 0.9555
Epoch 9/20
100/100 [==============================] - 12s 119ms/step - loss: 0.3314 - auc_1: 0.9598
Epoch 10/20
100/100 [==============================] - 12s 120ms/step - 

In [39]:
# Print classes
print(train_generator.class_indices)

{'benign': 0, 'malignant': 1, 'normal': 2}


In [31]:
# # Extracting the history of training and validation accuracy and loss
# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(1, len(acc) + 1)

# # Plotting training and validation accuracy
# plt.figure(figsize=(12, 5))
# plt.subplot(1, 2, 1)
# plt.plot(epochs, acc, 'bo', label='Training acc')
# # plt.plot(epochs, val_acc, 'b', label='Validation acc')
# plt.title('Training and Validation Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()

# # Plotting training and validation loss
# plt.subplot(1, 2, 2)
# plt.plot(epochs, loss, 'bo', label='Training loss')
# # plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()

# plt.tight_layout()
# plt.show()

# Extracting the history of training and validation AUC and loss
auc = history.history['auc']
val_auc = history.history['val_auc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(auc) + 1)

# Plotting training and validation AUC
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, auc, 'bo', label='Training AUC')
plt.plot(epochs, val_auc, 'b', label='Validation AUC')
plt.title('Training and Validation AUC')
plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.legend()

# Plotting training and validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

KeyError: 'auc'

In [40]:
validation_generator = test_datagen.flow_from_directory(
        validation_path,  # Assuming validation_dir is defined
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')  # Change for multi-class

test_loss, test_auc = model.evaluate(validation_generator)
print(f"Test AUC: {test_auc*100:.1f}%")
print(f"Test Loss: {test_loss:.1f}")

Found 965 images belonging to 3 classes.
31/31 [==============================] - 2s 56ms/step - loss: 0.3969 - auc_1: 0.9581
Test AUC: 95.8%
Test Loss: 0.4


In [41]:
# Save the entire model as a `.keras` zip archive.
model.save('model.keras')

In [42]:
new_model = tf.keras.models.load_model('model.keras')

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 17, 17, 128)     

In [ ]:
# Assuming the use of a pre-trained model like MobileNet for transfer learning
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False  # Freeze the base model

# Add custom layers on top for our specific problem
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout
predictions = Dense(3, activation='softmax')(x)  # Change for three classes

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,  # Depends on your dataset size
      epochs=15,
      validation_data=validation_generator,
      validation_steps=50)  # Depends on your dataset size